# Imports

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preprocessing

In [15]:
# test
from datasets import load_dataset

for name in ["inpainting.zip", "insight.zip", "text2img.zip", "wiki.zip"]:
    print(f"--- {name} ---")
    ds = load_dataset("OpenRL/DeepFakeFace", data_files=name, split="train", streaming=True)
    for i, item in enumerate(ds):
        print(item)
        if i == 2:  # Show only the first 3 samples
            break

--- inpainting.zip ---
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x168C08CB0>, 'label': 0}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x161E3D040>, 'label': 0}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x168C08C50>, 'label': 0}
--- insight.zip ---
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x168711580>, 'label': 0}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x168C080B0>, 'label': 0}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x168C0B1D0>, 'label': 0}
--- text2img.zip ---
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x161E3C380>, 'label': 0}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x168712A80>, 'label': 0}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x161E87A10>, 'label': 0}
--- wiki.zip ---
{